In [1]:
from collections import OrderedDict
from typing import List, Tuple, Union, Optional

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr as fl
import numpy as np
import torch
import glob
import os
from flwr.common import ndarrays_to_parameters
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context, FitRes, Parameters, Scalar
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {fl.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

d:\Anaconda\envs\TMenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-15 14:10:45,267	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Training on cuda
Flower 1.19.0 / PyTorch 2.5.1+cu121


In [2]:
NUM_CLIENTS = 2
BATCH_SIZE = 256
NUM_ROUNDS = 200

from test_flwr import get_all_vocab, split_data
vocab = get_all_vocab(["datasets/20NG"])
datasets = split_data(dir = "datasets/20NG", num_split=NUM_CLIENTS, vocab = vocab, batch_size= BATCH_SIZE)

train_size:  11314
test_size:  7532
vocab_size:  5000
average length: 110.543


parsing texts: 100%|██████████| 5657/5657 [00:00<00:00, 11119.96it/s]


In [3]:
from model.ETM import ETM
from trainer.basic_trainer import BasicTrainer



In [4]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [5]:
from data.basic_dataset import RawDataset
class FlowerClient(NumPyClient):
  def __init__(self, net, dataset : RawDataset, id):
    self.net = net
    self.dataset = dataset
    self.trainer = BasicTrainer(net, dataset, epochs = 1, log_interval=10, device = DEVICE, save_model = True, save_interval=NUM_ROUNDS)
    self.id = id
    self.save_dir = "model_parameters/"
    self.round_id = 0
    self.total_round = NUM_ROUNDS

  # return the current local model parameters
  def get_parameters(self, config):
    return get_parameters(self.net)

  # receive global parameter, train, return updated model to server
  def fit(self, parameters, config):
    set_parameters(self.net, parameters)
    self.trainer.train(model_name = f"ETM_Client{self.id}")

    return get_parameters(self.net), len(self.dataset.train_texts), {}

  # receive global parameter, evaluate model from local's data, return the evaluation result
  def evaluate(self, parameters, config):
    set_parameters(self.net, parameters)
    loss, acc = -1, -1
    return float(loss), 1, {"accuracy":float(acc)}


test = FlowerClient(ETM(len(vocab)), datasets[0], 0)

In [6]:
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = ETM(len(vocab)).to(DEVICE)

    # Load data
    partition_id = context.node_config["partition-id"]
    dataset = datasets[partition_id]

    # Create a single Flower client representing a single organization
    return FlowerClient(net, dataset, partition_id).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [7]:
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: list[tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: list[Union[tuple[ClientProxy, FitRes], BaseException]],
    ) -> tuple[Optional[Parameters], dict[str, Scalar]]:

        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(
            server_round, results, failures
        )

        if aggregated_parameters is not None:
            # Convert `Parameters` to `list[np.ndarray]`
            aggregated_ndarrays: list[np.ndarray] = fl.common.parameters_to_ndarrays(
                aggregated_parameters
            )

            # Save aggregated_ndarrays to disk
            if server_round % 10 == 0:
                print(f"Saving round {server_round} aggregated_ndarrays...")
                np.savez(f"model_parameters/model_round_{server_round}.npz", *aggregated_ndarrays)

        return aggregated_parameters, aggregated_metrics


# Create strategy and pass into ServerApp
def server_fn(context):
    strategy = SaveModelStrategy(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=NUM_CLIENTS,
        min_available_clients=NUM_CLIENTS,
    )
    config = ServerConfig(num_rounds=NUM_ROUNDS)
    return ServerAppComponents(strategy=strategy, config=config)


server = ServerApp(server_fn=server_fn)

In [8]:
def get_latest_server_model(net):
    list_of_files = [fname for fname in glob.glob("model_parameters/model_round_*")]
    latest_round_file = max(list_of_files, key=os.path.getctime)
    print("Loading pre-trained model from: ", latest_round_file)
    
    # Load NumPy arrays from .npz file
    with np.load(latest_round_file) as data:
        arrays = [data[f'arr_{i}'] for i in range(len(data.files))]
    
    # Convert to PyTorch state_dict
    state_dict = {k: torch.from_numpy(v) for k, v in zip(net.state_dict().keys(), arrays)}
    net.load_state_dict(state_dict)
    
    # Convert to Flower Parameters
    state_dict_ndarrays = [v.cpu().numpy() for v in net.state_dict().values()]
    parameters = fl.common.ndarrays_to_parameters(state_dict_ndarrays)
    return parameters

In [9]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [10]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
    verbose_logging=False
)

INFO :      Starting Flower ServerApp, config: num_rounds=200, no round_timeout
INFO :      


INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


(ClientAppActor pid=14088) Client's model: ETM_Client0
(ClientAppActor pid=14088) Epoch: 000 | Loss: 1969.922607421875


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)


(ClientAppActor pid=14088) Client's model: ETM_Client1
(ClientAppActor pid=14088) Epoch: 000 | Loss: 1933.206298828125


...

INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [ROUND 200]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


(ClientAppActor pid=14088) Client's model: ETM_Client0
(ClientAppActor pid=14088) Epoch: 000 | Loss: 754.8633422851562


INFO :      aggregate_fit: received 2 results and 0 failures


(ClientAppActor pid=14088) Client's model: ETM_Client1
(ClientAppActor pid=14088) Epoch: 000 | Loss: 739.5944213867188
Saving round 200 aggregated_ndarrays...


INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 200 round(s) in 882.81s
INFO :      	History (loss, distributed):
INFO :      		round 1: -1.0
INFO :      		round 2: -1.0
INFO :      		round 3: -1.0
INFO :      		round 4: -1.0
INFO :      		round 5: -1.0
INFO :      		round 6: -1.0
INFO :      		round 7: -1.0
INFO :      		round 8: -1.0
INFO :      		round 9: -1.0
INFO :      		round 10: -1.0
INFO :      		round 11: -1.0
INFO :      		round 12: -1.0
INFO :      		round 13: -1.0
INFO :      		round 14: -1.0
INFO :      		round 15: -1.0
INFO :      		round 16: -1.0
INFO :      		round 17: -1.0
INFO :      		round 18: -1.0
INFO :      		round 19: -1.0
INFO :      		round 20: -1.0
INFO :      		round 21: -1.0
INFO :      		round 22: -1.0
INFO :      		round 23: -1.0
INFO :      		round 24: -1.0
INFO :      		round 25: -1.0
INFO :      		round 26

In [11]:
from utils._utils import get_top_words
net = ETM(len(vocab))
test = get_latest_server_model(net)
beta = net.get_beta().detach().cpu().numpy()
topwords = get_top_words(beta, vocab, 15, verbose=True)

Loading pre-trained model from:  model_parameters\model_round_200.npz
Topic 0: organization lines long real group things idea general small video robert service exactly rate coming
Topic 1: right god second things play source word years works michael machine israeli period rest road
Topic 2: going game news files department server original keywords position military model business laws andrew die
Topic 3: use probably getting available newsreader needed apple non legal hold heard commercial internal necessarily mention
Topic 4: sure window given message man check came faith death history society case way medical included
Topic 5: writes run company user large disclaimer ram online soldiers definition transfer define story nature special
Topic 6: organization file question windows able start address canada summary light night gary reports guys cleveland
Topic 7: number example true evidence pretty type took talking asked york anti today chicago present bought
Topic 8: technology mac cli

In [12]:
trainer = BasicTrainer(net, datasets[0])

In [13]:
net.to(DEVICE)

ETM(
  (encoder1): Sequential(
    (0): Linear(in_features=5000, out_features=800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=800, out_features=800, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
  )
  (fc21): Linear(in_features=800, out_features=50, bias=True)
  (fc22): Linear(in_features=800, out_features=50, bias=True)
)

In [14]:
########################### test new documents ####################################
from data.preprocess import Preprocess

preprocess = Preprocess()

new_docs = [
    "This is a new document about space, including words like space, satellite, launch, orbit.",
    "This is a new document about Microsoft Windows, including words like windows, files, dos."
]

parsed_new_docs, new_bow = preprocess.parse(new_docs, vocab)
print(new_bow.shape)

print(new_bow.toarray())
input = torch.as_tensor(new_bow.toarray(), device="cuda").float()
print(input)
new_theta = trainer.test(input)

print(new_theta.argmax(1))
for x in new_theta.argmax(1):
    print(topwords[x])

parsing texts: 100%|██████████| 2/2 [00:00<00:00, 2002.05it/s]


(2, 5000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
[36  6]
space email sun including black taken lost defense federal tom baseball machines add received release
organization file question windows able start address canada summary light night gary reports guys cleveland
